In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import yfinance as yf
import time

C:\Users\hengs\Anaconda3\lib\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


## Assumptions

- Twitter: scraped from 5/9 - 5/10
- Reddit: scrapped from 18/8 - 17/9


- Mkt data: 4am-8pm (last record at 7.45pm): 
- Pre-market: 4am-9.30am; Post-market: 4pm-8pm


- Opening price
- round to nearest 15mins 
- non-trading hours: follow price when mkt closes (incld weekend, after hours, PH)
- for NAN prices, use previous time peiod price
- to have variable to state closing/opening hours

In [2]:
# Parameters
roundTo = 15
stock_list = ['GME', 'AMC', 'MVIS']

## Twitter data-processing

In [3]:
gme = pd.read_csv('gme_clean_sentiment.csv', index_col=0)
gme['stock'] = 'GME'
amc = pd.read_csv('amc_clean_sentiment.csv', index_col=0)
amc['stock'] = 'AMC'
mvis = pd.read_csv('mvis_clean_sentiment.csv', index_col=0)
mvis['stock'] = 'MVIS'

# add all twitter data into one df for further cleaning
twitter = pd.DataFrame()
twitter = twitter.append(gme)
twitter = twitter.append(amc)
twitter = twitter.append(mvis)
twitter = twitter.reset_index()

# some values in date col are not datetime. to remove them
twitter = twitter[~twitter['date'].apply(lambda x: len(x) != 25)]

twitter.head() #105691 rows

,date,content,Sentiment_textblob,Sentiment_vader,stock
0,2021-09-18 23:56:45+00:00,confirms drs protect share phantom share put d...,-0.200,0.7717,GME
1,2021-09-18 23:53:47+00:00,aggressivevolatile high short squeeze stock po...,0.032,-0.3182,GME
2,2021-09-18 23:50:11+00:00,spacexs private inspiration4 mission splash sa...,0.250,0.4939,GME
3,2021-09-18 23:50:00+00:00,alert run gain daily alert one time fee beat t...,0.000,0.7783,GME
4,2021-09-18 23:48:38+00:00,stop,0.000,-0.2960,GME


In [4]:
# clean datetime col and convert to UTC - 4
twitter['cleaned_date'] = twitter['date'].apply(lambda x: datetime.strptime(x[:-6], '%Y-%m-%d %H:%M:%S') - timedelta(hours=4)) 

# rounds time to nearest roundTo minutes
def roundTime(dt, roundTo):
    
    mins = dt.minute
    if mins % roundTo < roundTo /2:
        rounded_dt = dt - timedelta(minutes=mins % roundTo)
    else:
        rounded_dt = dt + timedelta(minutes=  roundTo - mins % roundTo)
    rounded_dt = rounded_dt.replace(second=0)
    return rounded_dt

twitter['cleaned_date'] = twitter['cleaned_date'].apply(lambda x: roundTime(x, roundTo))

twitter = twitter[['cleaned_date', 'content', 'stock']].reset_index(drop=True)
twitter['source'] = 'Twitter'
twitter.head()

,cleaned_date,content,stock,source
0,2021-09-18 20:00:00,confirms drs protect share phantom share put d...,GME,Twitter
1,2021-09-18 20:00:00,aggressivevolatile high short squeeze stock po...,GME,Twitter
2,2021-09-18 19:45:00,spacexs private inspiration4 mission splash sa...,GME,Twitter
3,2021-09-18 19:45:00,alert run gain daily alert one time fee beat t...,GME,Twitter
4,2021-09-18 19:45:00,stop,GME,Twitter


## Reddit Data-processing

In [19]:
reddit = pd.read_csv('Meme Stock2.0.csv', index_col=0) #4704 rows

reddit = reddit.reset_index()
reddit.head()

,date,body,split_text,stock,Clean
0,15/9/2021 14:27,AMC baby buy that dip before the big bounce 🍿🚀...,AMC baby buy dip big bounce,AMC,amc baby buy dip big bounce
1,15/9/2021 14:38,"So glad I got in on AMC during Dip, gonna be m...",So glad I got AMC Dip gonna mooning day babyyy...,I AMC,glad get amc dip mooning day babyyy img
2,15/9/2021 14:06,Once the news stops talking about AMC and meme...,Once news stops talking AMC meme stocks going 10,AMC,once news stop talk amc meme stock
3,15/9/2021 14:08,"Yes the ""hedgies"" are monitoring your 10 share...",Yes hedgies monitoring 10 shares AMC spend mil...,AMC,yes hedgie monitoring share amc spend million ...
4,15/9/2021 14:45,Thoughts on MVIS?,Thoughts MVIS,MVIS,thought mvis


In [20]:
def generate_stock_list(stock_col):
    stocks = stock_col.split()
    filtered_stocks = [s for s in stocks if s in stock_list]
    return filtered_stocks

# remove unrelated stocks in stock col
reddit['stock'] = reddit['stock'].apply(generate_stock_list) #4704 rows

# duplicate posts with multiple stocks
reddit = reddit.explode('stock') #5553 rows

# clean datetime and convert to UTC-4
reddit['cleaned_date'] = reddit['date'].apply(lambda x: datetime.strptime(x, '%d/%m/%Y %H:%M') - timedelta(hours=4))

# rounds time to nearest roundTo minutes
reddit['cleaned_date'] = reddit['cleaned_date'].apply(lambda x: roundTime(x, roundTo))

reddit = reddit[['cleaned_date', 'Clean', 'stock']].reset_index(drop=True)
reddit = reddit.rename(columns={'Clean': 'content'})
reddit['source'] = 'Reddit'

reddit.head()

,cleaned_date,content,stock,source
0,2021-09-15 10:30:00,amc baby buy dip big bounce,AMC,Reddit
1,2021-09-15 10:45:00,glad get amc dip mooning day babyyy img,AMC,Reddit
2,2021-09-15 10:00:00,once news stop talk amc meme stock,AMC,Reddit
3,2021-09-15 10:15:00,yes hedgie monitoring share amc spend million ...,AMC,Reddit
4,2021-09-15 10:45:00,thought mvis,MVIS,Reddit


## Combined df data-processing

In [21]:
# combine Reddit and twitter data
df = twitter.copy().append(reddit)

print(df['stock'].value_counts())
print(df['source'].value_counts())
df.head()

AMC     61767
GME     46797
MVIS     2674
Name: stock, dtype: int64
Twitter    105691
Reddit       5553
Name: source, dtype: int64


,cleaned_date,content,stock,source
0,2021-09-18 20:00:00,confirms drs protect share phantom share put d...,GME,Twitter
1,2021-09-18 20:00:00,aggressivevolatile high short squeeze stock po...,GME,Twitter
2,2021-09-18 19:45:00,spacexs private inspiration4 mission splash sa...,GME,Twitter
3,2021-09-18 19:45:00,alert run gain daily alert one time fee beat t...,GME,Twitter
4,2021-09-18 19:45:00,stop,GME,Twitter


In [22]:
df['Day'] = df['cleaned_date'].apply(lambda x: x.day)
df['Month'] = df['cleaned_date'].apply(lambda x: x.month)
df['Hour'] = df['cleaned_date'].apply(lambda x: x.hour)
df['Minute'] = df['cleaned_date'].apply(lambda x: x.minute)
df['DayofWeek'] = df['cleaned_date'].apply(lambda x: x.weekday() +1)

def get_dummy_date(row):
   # returns same col as cleaned_date only if its within trading hours (4am-8pm). Else, return last trading time (1945 previous day)
    # Purpose: for join
    date = row['cleaned_date']
    day = date.day
    month = date.month
    hour = date.hour
    minute = date.minute
    dayOfWeek = date.weekday() +1  # range will be 1-7
    
    # Weekends
    if dayOfWeek in [6,7]:
        diff_day = dayOfWeek - 5
        
        date = date - timedelta(days=diff_day)
        date = date.replace(hour=19, minute=45)
    
    # Weekdays
    else:
        if hour >=20:
            date = date.replace(hour=19, minute=45)
        elif hour < 4:
            if dayOfWeek == 1: #Mon 2am goes to previous Fri 1945
                date = date - timedelta(days=3)
            else:
                date = date - timedelta(days=1)
            date = date.replace(hour=19, minute=45)
    
    if month == 9 and day == 6: 
        # account for 6 Sept: PH --> go to 3/9 1945
        date = date.replace(month=9, day=3, hour=19, minute=45)
        
    return date

def trading_hours(row):
    # 1 if weekdays 9.30-4pm (both inclusive), else 0
    if row['DayofWeek'] in [1,2,3,4,5]:
        if row['Hour'] == 9 and row['Minute'] >= 30:
            return 1
        if row['Hour'] > 9 and row['Hour'] <= 16:
            return 1
    return 0

def pre_trading(row):
    # 1 if weekdays 4am (included) -9.30am (excluded), else 0
    if row['DayofWeek'] in [1,2,3,4,5]:
        if row['Hour'] == 9 and row['Minute'] < 30:
            return 1
        if row['Hour'] >= 4 and row['Hour'] < 9:
            return 1
    return 0

def post_trading(row):
    # 1 if weekdays 4pm (excluded) - 8pm (included), else 0
    if row['DayofWeek'] in [1,2,3,4,5]:
        if row['Hour'] > 16 and row['Hour'] <= 20:
            return 1
    return 0

df['trading_hours'] = df.apply(trading_hours, axis=1)
df['pre_trading'] = df.apply(pre_trading, axis=1)
df['post_trading'] = df.apply(post_trading, axis=1)
df['dummy_date'] = df.apply(get_dummy_date, axis=1)

df.head()

,cleaned_date,content,stock,source,Day,Month,Hour,Minute,DayofWeek,trading_hours,pre_trading,post_trading,dummy_date
0,2021-09-18 20:00:00,confirms drs protect share phantom share put d...,GME,Twitter,18,9,20,0,6,0,0,0,2021-09-17 19:45:00
1,2021-09-18 20:00:00,aggressivevolatile high short squeeze stock po...,GME,Twitter,18,9,20,0,6,0,0,0,2021-09-17 19:45:00
2,2021-09-18 19:45:00,spacexs private inspiration4 mission splash sa...,GME,Twitter,18,9,19,45,6,0,0,0,2021-09-17 19:45:00
3,2021-09-18 19:45:00,alert run gain daily alert one time fee beat t...,GME,Twitter,18,9,19,45,6,0,0,0,2021-09-17 19:45:00
4,2021-09-18 19:45:00,stop,GME,Twitter,18,9,19,45,6,0,0,0,2021-09-17 19:45:00


## Get stock prices + data processing

In [9]:
# stock_string = " ".join(stock_list)

# mkt_data_raw = yf.download(tickers=stock_string,
#                  interval='5m',
#                  start='2021-08-19', 
#                  end='2021-10-05',
#                  group_by='ticker',
#                  prepost=True)  #prepost=True for pre- and post-market hours

# mkt_data_raw = mkt_data_raw.reset_index()
# mkt_data_raw.head()


In [10]:
# mkt_data = mkt_data_raw.copy()
# mkt_data['cleaned_date'] = mkt_data['Datetime'].apply(lambda x: x.replace(tzinfo=None))
# mkt_data.to_csv('market_data_v2(5m interval).csv')

In [11]:
# # split prices by ticker. To use opening price
# import warnings
# warnings.filterwarnings('ignore')

# amc_data = mkt_data[['cleaned_date']]
# amc_data['Open'] = mkt_data['AMC']['Open']
# amc_data.columns = [x[0] for x in amc_data.columns]

# mvis_data = mkt_data[['cleaned_date']]
# mvis_data['Open'] = mkt_data['MVIS']['Open']
# mvis_data.columns = [x[0] for x in mvis_data.columns]

# gme_data = mkt_data[['cleaned_date']]
# gme_data['Open'] = mkt_data['GME']['Open']
# gme_data.columns = [x[0] for x in gme_data.columns]

# stock_mapping = {'AMC': amc_data,
#                 'MVIS': mvis_data,
#                 'GME': gme_data}

# amc_data.to_csv('amc_prices_v2.csv')
# mvis_data.to_csv('mvis_prices_v2.csv')
# gme_data.to_csv('gme_prices_v2.csv')

In [12]:
amc_data = pd.read_csv('amc_prices.csv', index_col=0)
mvis_data = pd.read_csv('mvis_prices.csv', index_col=0)
gme_data = pd.read_csv('gme_prices.csv', index_col=0)

In [23]:
# prices from 16/8 to 5/10
amc_data = pd.read_csv('amc_prices.csv', index_col=0)
mvis_data = pd.read_csv('mvis_prices.csv', index_col=0)
gme_data = pd.read_csv('gme_prices.csv', index_col=0)

for data in [amc_data, mvis_data, gme_data]:
    data['Open'] = data['Open'].fillna(method='ffill')
    data['cleaned_date'] = data['cleaned_date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
    
stock_mapping = {'AMC': amc_data,
                'MVIS': mvis_data,
                'GME': gme_data}

In [24]:
# join prices to df
result = pd.DataFrame()
for stock in stock_list:
    stock_posts = df[df['stock'] == stock]
    stock_price = stock_mapping[stock]
    
    combined_df = stock_posts.merge(stock_price, left_on='dummy_date', right_on = 'cleaned_date', how='left')
    result = pd.concat([result, combined_df], axis=0)
    
result = result.rename(columns={'cleaned_date_x': 'date',
                               'Open': 'price'})
result = result.drop(columns=['dummy_date', 'cleaned_date_y'])

result.head()

,date,content,stock,source,Day,Month,Hour,Minute,DayofWeek,trading_hours,pre_trading,post_trading,price
0,2021-09-18 20:00:00,confirms drs protect share phantom share put d...,GME,Twitter,18,9,20,0,6,0,0,0,205.01
1,2021-09-18 20:00:00,aggressivevolatile high short squeeze stock po...,GME,Twitter,18,9,20,0,6,0,0,0,205.01
2,2021-09-18 19:45:00,spacexs private inspiration4 mission splash sa...,GME,Twitter,18,9,19,45,6,0,0,0,205.01
3,2021-09-18 19:45:00,alert run gain daily alert one time fee beat t...,GME,Twitter,18,9,19,45,6,0,0,0,205.01
4,2021-09-18 19:45:00,stop,GME,Twitter,18,9,19,45,6,0,0,0,205.01


In [25]:
# result.to_csv('results_v2.csv', index=0)

Observations: 9.15am prices are missing, but is imputed with previous value

In [26]:
reddit1 = pd.read_csv('reddit_sentiment.csv')

In [28]:
reddit1 = reddit1[reddit1['source'] == 'Reddit']

In [32]:
reddit1.head()

,date,content,stock,source,Day,Month,Hour,Minute,DayofWeek,trading_hours,pre_trading,post_trading,price,Sentiment_textblob,Sentiment_vader
41506,15/9/2021 7:15,boy tsla 700p look dicey ampx200b gme save cel...,GME,Reddit,15.0,9.0,7.0,15.0,3.0,0.0,1.0,0.0,197.980000,0.000000,0.4939
41507,15/9/2021 14:30,opad run like gme friday those option friday w...,GME,Reddit,15.0,9.0,14.0,30.0,3.0,1.0,0.0,0.0,204.287506,0.300000,0.5267
41508,16/9/2021 13:15,people use sub steal retard money postgme mayb...,GME,Reddit,16.0,9.0,13.0,15.0,4.0,1.0,0.0,0.0,213.389999,0.039286,-0.2732
41509,17/9/2021 7:30,guess talk gme today,GME,Reddit,17.0,9.0,7.0,30.0,5.0,0.0,1.0,0.0,207.630000,0.000000,0.0000
41510,17/9/2021 8:45,meme fly week daddy gme rise,GME,Reddit,17.0,9.0,8.0,45.0,5.0,0.0,1.0,0.0,206.700000,0.800000,0.0000


In [16]:
a1 = result[result['price'].isnull()]
# a1 = a1[~(a1['Hour']==9) & ~(a1['Minute']==15)]
a1.groupby(['Month','Day','DayofWeek','Hour']).size().head(50)

Month  Day  DayofWeek  Hour
8      27   5          9       31
9      7    2          0       36
                       1       26
                       2       23
                       3       19
dtype: int64

In [17]:
a2 = a[(a['Month'] == 9) & ~(a['Hour'].isin([9,18]))]
a2.head(50)

NameError: name 'a' is not defined

In [ ]:
print(result['stock'].value_counts())
print(a['stock'].value_counts())


